In [9]:
%%html
<script>
MathJax = {
  tex: {
    inlineMath: [['$', '$']]
  },
  svg: {
    fontCache: 'global'
  }
};
</script>
<script type="text/javascript" id="MathJax-script" async
  src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-svg.js">
</script>

Neighbour-joining trees (NJT) are a useful tool for exploratory analysis of genetic population structure, which I've [written about recently](https://alimanfoo.github.io/2024/09/05/kenya-coluzzii-njt.html). Previously I've used [biotite](https://www.biotite-python.org/latest/index.html) to compute NJT for hundreds of individuals, but recently I was trying to build some bigger trees with thousands of individuals and things started to take a long time to run. I then learned of course that the canonical neighbour-joining algorithm is $O(n^3)$ meaning that speed of computation scales with the number of samples cubed. This happens because neighbour-joining is an interative algorithm, where each iteration involves searching a distance matrix for the pair of nearest neighbours.

I spent some time looking at the biotite implementation which uses the canonical algorithm that fully searches the distance matrix at each iteration and is written in Cython. I saw a few opportunities to make the implementation more efficient by avoiding some unnecessary work. I also read around a little and found a [paper by Martin Simonsen et al. from 2008](https://pure.au.dk/ws/files/19821675/rapidNJ.pdf) describing a rapid neighbour-joining algorithm which uses some heuristics to avoid doing a full search of the distance matrix in each iteration. There is a [C++ implementation of the rapid algorithm](https://github.com/somme89/rapidNJ) but I'd really like something efficient that I can call directly from Python.

After a bit of prototyping I was fairly confident I could at least do an implementation of the canonical neighbour-joining algorithm using [numba](https://numba.pydata.org/) that was pretty fast. After a bit more tuning I managed to get around a 10x speedup over biotite. E.g., for a dataset of 3000 individuals, biotite took ~55s and my implementation took ~6s. I decided this was worth packaging and so created a [new Python packaged called "anjl"](https://alimanfoo.github.io/anjl/) which stands for "A neighbour-joining library". 